In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

/kaggle/input/train_V2.csv
/kaggle/input/test_V2.csv
/kaggle/input/sample_submission_V2.csv


**Data fields**
- DBNOs - Number of enemy players knocked.
- assists - Number of enemy players this player damaged that were killed by teammates.
- boosts - Number of boost items used.
- damageDealt - Total damage dealt. Note: Self inflicted damage is subtracted.
- headshotKills - Number of enemy players killed with headshots.
- heals - Number of healing items used.
- Id - Player’s Id
- killPlace - Ranking in match of number of enemy players killed.
- killPoints - Kills-based external ranking of player. (Think of this as an Elo ranking where only kills matter.) If there is a value other than -1 in rankPoints, then any 0 in killPoints should be treated as a “None”.
- killStreaks - Max number of enemy players killed in a short amount of time.
- kills - Number of enemy players killed.
- longestKill - Longest distance between player and player killed at time of death. This may be misleading, as downing a player and driving away may lead to a large longestKill stat.
- matchDuration - Duration of match in seconds.
- matchId - ID to identify match. There are no matches that are in both the training and testing set.
- matchType - String identifying the game mode that the data comes from. The standard modes are “solo”, “duo”, “squad”, “solo-fpp”, “duo-fpp”, and “squad-fpp”; other modes are from events or custom matches.
- rankPoints - Elo-like ranking of player. This ranking is inconsistent and is being deprecated in the API’s next version, so use with caution. Value of -1 takes place of “None”.
- revives - Number of times this player revived teammates.
- rideDistance - Total distance traveled in vehicles measured in meters.
- roadKills - Number of kills while in a vehicle.
- swimDistance - Total distance traveled by swimming measured in meters.
- teamKills - Number of times this player killed a teammate.
- vehicleDestroys - Number of vehicles destroyed.
- walkDistance - Total distance traveled on foot measured in meters.
- weaponsAcquired - Number of weapons picked up.
- winPoints - Win-based external ranking of player. (Think of this as an Elo ranking where only winning matters.) If there is a value other than -1 in rankPoints, then any 0 in winPoints should be treated as a “None”.
- groupId - ID to identify a group within a match. If the same group of players plays in different matches, they will have a different groupId each time.
- numGroups - Number of groups we have data for in the match.
- maxPlace - Worst placement we have data for in the match. This may not match with numGroups, as sometimes the data skips over placements.
- winPlacePerc - The target of prediction. This is a percentile winning placement, where 1 corresponds to 1st place, and 0 corresponds to last place in the match. It is calculated off of maxPlace, not numGroups, so it is possible to have missing chunks in a match.

In [2]:
train_path = "../input/train_V2.csv"
train_data = pd.read_csv(train_path, index_col="Id")
train_data.head()

,groupId,matchId,assists,boosts,damageDealt,DBNOs,headshotKills,heals,killPlace,killPoints,...,revives,rideDistance,roadKills,swimDistance,teamKills,vehicleDestroys,walkDistance,weaponsAcquired,winPoints,winPlacePerc
Id,,,,,,,,,,,,,,,,,,,,,
7f96b2f878858a,4d4b580de459be,a10357fd1a4a91,0,0,0.00,0,0,0,60,1241,...,0,0.0000,0,0.00,0,0,244.80,1,1466,0.4444
eef90569b9d03c,684d5656442f9e,aeb375fc57110c,0,0,91.47,0,0,0,57,0,...,0,0.0045,0,11.04,0,0,1434.00,5,0,0.6400
1eaf90ac73de72,6a4a42c3245a74,110163d8bb94ae,1,0,68.00,0,0,0,47,0,...,0,0.0000,0,0.00,0,0,161.80,2,0,0.7755
4616d365dd2853,a930a9c79cd721,f1f1f4ef412d7e,0,0,32.90,0,0,0,75,0,...,0,0.0000,0,0.00,0,0,202.70,3,0,0.1667
315c96c26c9aac,de04010b3458dd,6dc8ff871e21e6,0,0,100.00,0,0,0,45,0,...,0,0.0000,0,0.00,0,0,49.75,2,0,0.1875


In [3]:
test_path = "../input/test_V2.csv"
test_data = pd.read_csv(test_path, index_col="Id")
test_data.head()

,groupId,matchId,assists,boosts,damageDealt,DBNOs,headshotKills,heals,killPlace,killPoints,...,rankPoints,revives,rideDistance,roadKills,swimDistance,teamKills,vehicleDestroys,walkDistance,weaponsAcquired,winPoints
Id,,,,,,,,,,,,,,,,,,,,,
9329eb41e215eb,676b23c24e70d6,45b576ab7daa7f,0,0,51.46,0,0,0,73,0,...,1500,0,0.0,0,0.0,0,0,588.0,1,0
639bd0dcd7bda8,430933124148dd,42a9a0b906c928,0,4,179.10,0,0,2,11,0,...,1503,2,4669.0,0,0.0,0,0,2017.0,6,0
63d5c8ef8dfe91,0b45f5db20ba99,87e7e4477a048e,1,0,23.40,0,0,4,49,0,...,1565,0,0.0,0,0.0,0,0,787.8,4,0
cf5b81422591d1,b7497dbdc77f4a,1b9a94f1af67f1,0,0,65.52,0,0,0,54,0,...,1465,0,0.0,0,0.0,0,0,1812.0,3,0
ee6a295187ba21,6604ce20a1d230,40754a93016066,0,4,330.20,1,2,1,7,0,...,1480,1,0.0,0,0.0,0,0,2963.0,4,0


In [4]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 4446966 entries, 7f96b2f878858a to 0d8e7ed728b6fd
Data columns (total 28 columns):
groupId            object
matchId            object
assists            int64
boosts             int64
damageDealt        float64
DBNOs              int64
headshotKills      int64
heals              int64
killPlace          int64
killPoints         int64
kills              int64
killStreaks        int64
longestKill        float64
matchDuration      int64
matchType          object
maxPlace           int64
numGroups          int64
rankPoints         int64
revives            int64
rideDistance       float64
roadKills          int64
swimDistance       float64
teamKills          int64
vehicleDestroys    int64
walkDistance       float64
weaponsAcquired    int64
winPoints          int64
winPlacePerc       float64
dtypes: float64(6), int64(19), object(3)
memory usage: 983.9+ MB


In [5]:
columns = train_data.columns
for column in columns:
    print(f"{column}: {train_data[column].isna().sum()}")

groupId: 0
matchId: 0
assists: 0
boosts: 0
damageDealt: 0
DBNOs: 0
headshotKills: 0
heals: 0
killPlace: 0
killPoints: 0
kills: 0
killStreaks: 0
longestKill: 0
matchDuration: 0
matchType: 0
maxPlace: 0
numGroups: 0
rankPoints: 0
revives: 0
rideDistance: 0
roadKills: 0
swimDistance: 0
teamKills: 0
vehicleDestroys: 0
walkDistance: 0
weaponsAcquired: 0
winPoints: 0
winPlacePerc: 1


since there is only one row with null value, we can safely drop that row

In [6]:
data = train_data.copy()
data = data.dropna()

In [7]:
columns = data.columns
for column in columns:
    print(f"{column}: {data[column].isna().sum()}")

groupId: 0
matchId: 0
assists: 0
boosts: 0
damageDealt: 0
DBNOs: 0
headshotKills: 0
heals: 0
killPlace: 0
killPoints: 0
kills: 0
killStreaks: 0
longestKill: 0
matchDuration: 0
matchType: 0
maxPlace: 0
numGroups: 0
rankPoints: 0
revives: 0
rideDistance: 0
roadKills: 0
swimDistance: 0
teamKills: 0
vehicleDestroys: 0
walkDistance: 0
weaponsAcquired: 0
winPoints: 0
winPlacePerc: 0


In [8]:
num_data = data.select_dtypes(include='number')

In [9]:
num_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 4446965 entries, 7f96b2f878858a to 0d8e7ed728b6fd
Data columns (total 25 columns):
assists            int64
boosts             int64
damageDealt        float64
DBNOs              int64
headshotKills      int64
heals              int64
killPlace          int64
killPoints         int64
kills              int64
killStreaks        int64
longestKill        float64
matchDuration      int64
maxPlace           int64
numGroups          int64
rankPoints         int64
revives            int64
rideDistance       float64
roadKills          int64
swimDistance       float64
teamKills          int64
vehicleDestroys    int64
walkDistance       float64
weaponsAcquired    int64
winPoints          int64
winPlacePerc       float64
dtypes: float64(6), int64(19)
memory usage: 882.1+ MB


In [10]:
# plt.figure(figsize=(15, 15))
# sns.heatmap(data=num_data.corr(), annot=True, fmt='.1f')
# plt.show()

In [11]:
# plt.figure(figsize=(15, 15))
# sns.scatterplot(x=num_data.walkDistance, y=num_data.winPlacePerc)
# plt.show()

from above plot we can say that as the walking distance of a player increases, chances of getting better placement increases. But some players placement is good despite very less walking distance. This may be due to a player playing in squad or duo mode.

In [12]:
# plt.figure(figsize=(15, 15))
# sns.scatterplot(x=num_data.damageDealt, y=num_data.kills)
# plt.show()

In [13]:
# average number of kills
print(data.kills.mean())

0.9247835321393355


In [14]:
# maximum kills by a player
print(data.kills.max())

72


In [15]:
y = num_data.winPlacePerc
X = num_data.drop(columns='winPlacePerc')

In [16]:
test_X = test_data.select_dtypes(include='number')

In [17]:
from sklearn.tree import DecisionTreeRegressor

model = DecisionTreeRegressor()
model.fit(X, y)

DecisionTreeRegressor(criterion='mse', max_depth=None, max_features=None,
                      max_leaf_nodes=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      presort=False, random_state=None, splitter='best')

In [18]:
output = model.predict(test_X)

In [19]:
out = pd.DataFrame({'Id': test_X.index, 'winPlacePerc': output})

In [20]:
out.to_csv('submission.csv', index=False)